In [1]:
######### libraries ################
import pandas as pd
import numpy as np
from scipy import stats

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.externals import joblib
import os
import matplotlib as mpl
import matplotlib.pyplot as plt

In [2]:
############ functions ################
def scale_plot_size(factor=3.0):
    default_figsize = mpl.rcParamsDefault['figure.figsize']
    mpl.rcParams['figure.figsize'] = [val*factor for val in default_figsize]
scale_plot_size(1.5)

def rms(df):
    return np.sqrt(df.dot(df)/df.size)


In [3]:
######### reading file & preprocessing ##########
header = ["Hr", "Min","Sec","uSec", "X","Y"]

training_set_path = os.path.expanduser('~/Vibrational_analysis/data/IEEE_dataset/Learning_set/Bearing1_1')

dirList = os.listdir(os.path.expanduser(training_set_path))
dirList.sort()
n = 0

for sets in dirList: 
    if sets == 'temp_00001.csv':
        break
    df = pd.read_csv(training_set_path+"/"+sets, header=None)
    df.columns = header
    data = np.array(df,np.float64)
    columnx = data[:,4]
    columny = data[:,5]

    if n==0 :
        x = columnx
        y = columny

        kurtx = stats.kurtosis(columnx)
        kurty = stats.kurtosis(columny)

        rmsx = rms(columnx)
        rmsy = rms(columny)
        
        n = 1
    else:
        x= np.append(x,columnx)
        y= np.append(y,columny)

        kurtx = np.append(kurtx,stats.kurtosis(columnx))
        kurty = np.append(kurty,stats.kurtosis(columny))

        rmsx = np.append(rmsx,rms(columnx))
        rmsy = np.append(rmsy,rms(columny))


In [4]:
########### defining input variables ###############
n = kurtx.shape[0]

X = np.vstack ((kurtx,kurty,rmsx,rmsy))
threshold_1 = int(n*0.50)
threshold_2 = int(n*0.25)

output = np.arange (n-1,-1,-1,dtype=np.float64)
output = np.reshape(output,(kurtx.shape[0],1))

Y = np.where( output>threshold_1 , 0 , 1 )
Y[-threshold_2:] = 2

Xnm = preprocessing.normalize(X)
Xnm = Xnm.transpose()

X_train, X_test, Y_train, Y_test = train_test_split(Xnm, Y, test_size=0.2, random_state=69)